In [58]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [59]:
# !rm -r -R --recursive anchor

In [60]:
# # Install a particular version of `google-cloud-storage` because (oddly enough) 
# # the  version on Colab and GCP is old. A dependency error below is okay.
# !pip install -q google-cloud-storage==1.43.0

In [61]:
# # authenticate below for Google Storage access as needed
# from google.colab import auth
# auth.authenticate_user()

In [62]:
# # install ngrok to emulate public IP / address
# !wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
# !unzip -u ngrok-stable-linux-amd64.zip

In [63]:
# # TODO: sign up for an ngrok account
# # then put your ngrok token below, uncomment, and execute
# !./ngrok authtoken 2JVUEdhb58Kv6Mv7FuCIXzlGDOC_54KirAsbTYbEwJgqJ7qmp

In [64]:
# # install a ngrok python package and a version of flask that works with it in 
# # colab
# !pip -q install flask-ngrok
# !pip -q install flask==0.12.2
# !pip -q install flask_restful

# Run the app

In [65]:
# # you need to upload your implementation of search_app.py
# import search_frontend as se

In [66]:
# # uncomment the code below and execute to reload the module when you make 
# # changes to search_frontend.py (after you upload again).
# import importlib
# importlib.reload(se)

In [67]:
# from flask_ngrok import run_with_ngrok
# run_with_ngrok(se.app) 
# se.app.run()

# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [52]:
import json

with open('queries_train.json', 'rt') as f: 
  queries = json.load(f)

In [53]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [54]:
import requests
from time import time
url = 'http://34.136.225.57:8080'
# place the domain you got from ngrok or GCP IP below. 
# url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'
failed = []
qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    failed.append(q)
  
  qs_res.append((q, duration, ap))

In [ ]:
print("----------------- search -------------------")
print('Avg respone time: ' + str(sum([i[1] for i in qs_res])/len(qs_res)))
print('MAP@40: ' + str(sum([i[2] for i in qs_res])/len(qs_res)))

In [56]:
import requests
from time import time
url = 'http://34.136.225.57:8080'
# place the domain you got from ngrok or GCP IP below. 
# url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'
failed = []

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search_body', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    failed.append(q)
  
  qs_res.append((q, duration, ap))

In [ ]:
print("----------------- body -------------------")
print('Avg respone time: ' + str(sum([i[1] for i in qs_res])/len(qs_res)))
print('MAP@40: ' + str(sum([i[2] for i in qs_res])/len(qs_res)))

In [58]:
import requests
from time import time
url = 'http://34.136.225.57:8080'
# place the domain you got from ngrok or GCP IP below. 
# url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'
failed = []

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search_title', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    failed.append(q)
  
  qs_res.append((q, duration, ap))

In [ ]:
print("----------------- title -------------------")
print('Avg respone time: ' + str(sum([i[1] for i in qs_res])/len(qs_res)))
print('MAP@40: ' + str(sum([i[2] for i in qs_res])/len(qs_res)))

In [60]:
import requests
from time import time
url = 'http://34.136.225.57:8080'
# place the domain you got from ngrok or GCP IP below. 
# url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'
failed = []

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search_anchor', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    failed.append(q)
  
  qs_res.append((q, duration, ap))

In [ ]:
print("----------------- anchor -------------------")
print('Avg respone time: ' + str(sum([i[1] for i in qs_res])/len(qs_res)))
print('MAP@40: ' + str(sum([i[2] for i in qs_res])/len(qs_res)))